In [1]:
%cd ..

D:\SoftUni\Data Science\Project


### Imports

In [2]:
import pandas as pd
import numpy as np
from src import functions

### Reading datasets

In [3]:
drugs_distribution_data = pd.read_csv('data/drugs_distribution_by_country.csv')

In [4]:
drugs_distribution_data

,country,heroin_trade,cocaine_trade,cannabis_trade,synthetic_drug_trade
0,Turkey,8.0,4.0,5.0,5.5
1,Cabo Verde,4.0,7.0,5.0,4.5
2,South Africa,7.5,5.5,3.5,8.5
3,Colombia,5.0,9.5,8.0,5.0
4,Peru,3.5,9.0,5.5,4.0
...,...,...,...,...,...
188,Jamaica,3.0,6.0,9.0,3.0
189,St. Vincent and the Grenadines,1.0,7.5,7.5,1.0
190,Slovenia,4.0,4.5,5.0,5.5
191,St. Lucia,1.0,7.0,7.0,1.0


In [5]:
mental_disorders_data = pd.read_csv('data/mental_disorder_by_country.csv')

In [6]:
mental_disorders_data

,country,mental_disorders_per_100k
0,Afghanistan,17904.0
1,Angola,14517.0
2,Albania,12240.0
3,Andorra,16247.0
4,United Arab Emirates,13856.0
...,...,...
199,Zambia,13308.0
200,Zimbabwe,11882.0
201,Egypt,15361.0
202,Sudan,16525.0


### Preprocessing 'country' column

The goal is to merge the above dataframes by their common column - 'country'.

In [7]:
countries_in_drugs_data = np.sort(drugs_distribution_data['country'].unique())

In [8]:
len(countries_in_drugs_data)

193

In [9]:
countrues_in_disorders_data = np.sort(mental_disorders_data['country'].unique())

In [10]:
len(countrues_in_disorders_data)

204

Countries in 'mental_disorders_data' are 11 more than in 'drugs_distribution_data'. 
My approach here will be to drop the rows in each dataset which are not present in the other. At the other hand there are some differences between the spelling of the same countries in the two datasets, which requires additional prepocessing.

In [11]:
# Country names which are present in drugs dataset but not in the disorders dataset
different_names_in_drugs = np.setdiff1d(countries_in_drugs_data, countrues_in_disorders_data)

In [12]:
different_names_in_drugs

array(['Bolivia', 'Brunei', 'Congo, Dem. Rep.', 'Congo, Rep.',
       'Czech Republic', 'Iran', 'Korea, DPR', 'Korea, Rep.', 'Laos',
       'Liechtenstein', 'Moldova', 'Russia', 'St. Kitts and Nevis',
       'St. Lucia', 'St. Vincent and the Grenadines', 'Syria', 'Tanzania',
       'Turkey', 'United States', 'Venezuela', 'Vietnam'], dtype=object)

In [13]:
# Country names which are present in the disorders dataset but not in the drugs dataset
np.setdiff1d(countrues_in_disorders_data, countries_in_drugs_data)

array(['American Samoa', 'Bermuda', 'Bolivia (Plurinational State of)',
       'Brunei Darussalam', 'Congo', 'Cook Islands', 'Czechia',
       "Democratic People's Republic of Korea",
       'Democratic Republic of the Congo', 'Greenland', 'Guam',
       'Iran (Islamic Republic of)', "Lao People's Democratic Republic",
       'Niue', 'Northern Mariana Islands', 'Palestine', 'Puerto Rico',
       'Republic of Korea', 'Republic of Moldova', 'Russian Federation',
       'Saint Kitts and Nevis', 'Saint Lucia',
       'Saint Vincent and the Grenadines', 'Syrian Arab Republic',
       'Taiwan (Province of China)', 'Tokelau', 'Türkiye',
       'United Republic of Tanzania', 'United States Virgin Islands',
       'United States of America', 'Venezuela (Bolivarian Republic of)',
       'Viet Nam'], dtype=object)

We can observe that data for Liechtenstein as a country is present only in drugs dataframe. Also there are several countries that are present only in mental disorders dataset. The merge process will 'clean' all of them from the resulting dataframe. The only remain for preprocessing countries will be these with different spelled names. A little manual work of comparing the above arrays is required in order to create an array which contains all different spelled country names in disordes dataset. Then it will be easy to equalize them to the corresponding countries in drugs dataframe and do the merge after.

In [14]:
different_spelled_countries_in_disorders = np.array(['Bolivia (Plurinational State of)', 'Brunei Darussalam', 'Democratic Republic of the Congo', 'Congo', 'Czechia',
                    'Iran (Islamic Republic of)', 'Democratic People\'s Republic of Korea', 'Republic of Korea', 'Lao People\'s Democratic Republic',
                    'Republic of Moldova', 'Russian Federation','Saint Kitts and Nevis', 'Saint Lucia', 'Saint Vincent and the Grenadines',
                    'Syrian Arab Republic', 'United Republic of Tanzania', 'Türkiye','United States of America', 'Venezuela (Bolivarian Republic of)', 'Viet Nam'])

In [15]:
# Delete Liechtenstein in the array
index = np.argwhere(different_names_in_drugs == 'Liechtenstein')
different_names_in_drugs = np.delete(different_names_in_drugs, index)

Now I can create a dictionary with key - value pairs corresponding to all different spelled countries in both dataframes.

In [16]:
different_named_countries_dict = {k: v for k, v in zip(different_spelled_countries_in_disorders, different_names_in_drugs)}

In [17]:
different_named_countries_dict

{'Bolivia (Plurinational State of)': 'Bolivia',
 'Brunei Darussalam': 'Brunei',
 'Democratic Republic of the Congo': 'Congo, Dem. Rep.',
 'Congo': 'Congo, Rep.',
 'Czechia': 'Czech Republic',
 'Iran (Islamic Republic of)': 'Iran',
 "Democratic People's Republic of Korea": 'Korea, DPR',
 'Republic of Korea': 'Korea, Rep.',
 "Lao People's Democratic Republic": 'Laos',
 'Republic of Moldova': 'Moldova',
 'Russian Federation': 'Russia',
 'Saint Kitts and Nevis': 'St. Kitts and Nevis',
 'Saint Lucia': 'St. Lucia',
 'Saint Vincent and the Grenadines': 'St. Vincent and the Grenadines',
 'Syrian Arab Republic': 'Syria',
 'United Republic of Tanzania': 'Tanzania',
 'Türkiye': 'Turkey',
 'United States of America': 'United States',
 'Venezuela (Bolivarian Republic of)': 'Venezuela',
 'Viet Nam': 'Vietnam'}

The dictionary is used to change the misspelled country names in mental_disorders_data dataframe.

In [18]:
functions.rplace_col_values(different_named_countries_dict, mental_disorders_data, 'country')

D:\SoftUni\Data Science\Project\src\functions.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].replace(df[df[col] == key][col].values[0], value, inplace = True)


### Merging

Now I can merge the data from both dataframes to get a resulted dataframe with 192 observations(number of countries in drugs data - 1 for Liechtenstein) and 6 features.

In [19]:
drugs_disorders_data = drugs_distribution_data.merge(mental_disorders_data, on="country", how = 'inner')

In [20]:
drugs_disorders_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   country                    192 non-null    object 
 1   heroin_trade               192 non-null    float64
 2   cocaine_trade              192 non-null    float64
 3   cannabis_trade             192 non-null    float64
 4   synthetic_drug_trade       192 non-null    float64
 5   mental_disorders_per_100k  192 non-null    float64
dtypes: float64(5), object(1)
memory usage: 9.1+ KB


In [21]:
# Changing the order of columns so 'mental_disorders_per_100k' to be next to 'country'
drugs_disorders_data = drugs_disorders_data[['country', 'mental_disorders_per_100k', 'heroin_trade', 'cocaine_trade', 'cannabis_trade', 'synthetic_drug_trade']]

In [22]:
drugs_disorders_data

,country,mental_disorders_per_100k,heroin_trade,cocaine_trade,cannabis_trade,synthetic_drug_trade
0,Turkey,15281.0,8.0,4.0,5.0,5.5
1,Cabo Verde,13165.0,4.0,7.0,5.0,4.5
2,South Africa,13825.0,7.5,5.5,3.5,8.5
3,Colombia,13095.0,5.0,9.5,8.0,5.0
4,Peru,15486.0,3.5,9.0,5.5,4.0
...,...,...,...,...,...,...
187,Jamaica,15087.0,3.0,6.0,9.0,3.0
188,St. Vincent and the Grenadines,15380.0,1.0,7.5,7.5,1.0
189,Slovenia,12323.0,4.0,4.5,5.0,5.5
190,St. Lucia,15669.0,1.0,7.0,7.0,1.0


### Saving resulted dataset

In [23]:
drugs_disorders_data.to_csv('data/drugs_disorders_by_country.csv', index=False)